In [13]:
import numpy as np
from skimage.feature import hog, local_binary_pattern
from skimage.io import imread
from skimage.transform import resize
from skimage.color import rgb2gray
import pandas as pd
import os
import cv2

In [14]:
# ============================================================================
# CRICKET-OPTIMIZED AUGMENTED FEATURE EXTRACTION
# Tailored for cricket bat, ball, and stadium detection
# ============================================================================

from skimage.transform import rotate as sk_rotate, rescale, warp, AffineTransform
from skimage.util import random_noise
from skimage.exposure import adjust_gamma
from skimage.filters import gaussian

def apply_cricket_augmentations(image_rgb):
    """
    Apply cricket-specific augmentations optimized for bat, ball, and stadium detection.
    Returns list of (augmentation_name, augmented_image) tuples.
    """
    # Ensure float [0,1] range
    if image_rgb.dtype == np.uint8:
        img = image_rgb.astype(np.float32) / 255.0
    else:
        img = image_rgb.copy()

    augmentations = []
    
    # Original image
    augmentations.append(('original', img))

    # ===== CRITICAL AUGMENTATIONS =====
    
    # 1) HORIZONTAL FLIP - Essential for left/right-handed batsmen
    augmentations.append(('flip_h', np.flip(img, axis=1)))

    # 2) BRIGHTNESS - Day vs day-night matches, shadows on field
    for gamma in [0.7, 1.3]:
        try:
            aug_bright = adjust_gamma(img, gamma=gamma)
            augmentations.append((f'brightness_{gamma}', np.clip(aug_bright, 0, 1)))
        except Exception as e:
            print(f"Warning: Brightness {gamma} failed - {e}")

    # # 3) CONTRAST - Stadium lighting variations
    # for alpha in [0.8, 1.2]:
    #     try:
    #         contrast_img = np.clip((img - 0.5) * alpha + 0.5, 0, 1)
    #         augmentations.append((f'contrast_{alpha}', contrast_img))
    #     except Exception as e:
    #         print(f"Warning: Contrast {alpha} failed - {e}")

    # ===== HIGHLY IMPORTANT =====
    
    # 4) SMALL ROTATION - Camera angle variations (keep minimal)
    for angle in [-10, 10]:
        try:
            aug = sk_rotate(img, angle=angle, mode='edge', preserve_range=True)
            augmentations.append((f'rot_{angle}', np.clip(aug, 0, 1)))
        except Exception as e:
            print(f"Warning: Rotation {angle} failed - {e}")

    # # 5) COLOR JITTER - Different cameras and broadcast feeds
    # # Hue shift simulation via channel manipulation
    # try:
    #     hue_shift = img.copy()
    #     # Rotate RGB channels slightly
    #     hue_shift[:, :, 0] = np.clip(img[:, :, 0] * 1.1, 0, 1)  # R channel
    #     hue_shift[:, :, 2] = np.clip(img[:, :, 2] * 0.9, 0, 1)  # B channel
    #     augmentations.append(('color_jitter_warm', hue_shift))
        
    #     hue_shift_cool = img.copy()
    #     hue_shift_cool[:, :, 0] = np.clip(img[:, :, 0] * 0.9, 0, 1)
    #     hue_shift_cool[:, :, 2] = np.clip(img[:, :, 2] * 1.1, 0, 1)
    #     augmentations.append(('color_jitter_cool', hue_shift_cool))
    # except Exception as e:
    #     print(f"Warning: Color jitter failed - {e}")

    # 6) GAUSSIAN NOISE - Broadcast compression, low-light D/N matches
    # for var in [0.005, 0.01]:
    #     try:
    #         noisy = random_noise(img, mode='gaussian', var=var, clip=True)
    #         augmentations.append((f'gauss_noise_{var}', np.clip(noisy, 0, 1)))
    #     except Exception as e:
    #         print(f"Warning: Gaussian noise failed - {e}")

    # ===== CRICKET-SPECIFIC =====
    
    # # 7) RANDOM CROP - Partial stadium view, zoomed action shots
    # for crop_ratio in [0.85, 0.90]:
    #     try:
    #         oh, ow = img.shape[:2]
    #         crop_h = int(oh * crop_ratio)
    #         crop_w = int(ow * crop_ratio)
            
    #         # Random crop position
    #         y_start = np.random.randint(0, oh - crop_h + 1) if oh > crop_h else 0
    #         x_start = np.random.randint(0, ow - crop_w + 1) if ow > crop_w else 0
            
    #         cropped = img[y_start:y_start+crop_h, x_start:x_start+crop_w]
    #         # Resize back to original size
    #         cropped_resized = resize(cropped, (oh, ow), anti_aliasing=True)
    #         augmentations.append((f'crop_{crop_ratio}', np.clip(cropped_resized, 0, 1)))
    #     except Exception as e:
    #         print(f"Warning: Random crop {crop_ratio} failed - {e}")

    # 8) CUTOUT/RANDOM ERASING - Occlusion by players/umpires
    # try:
    #     cutout_img = img.copy()
    #     oh, ow = cutout_img.shape[:2]
    #     # Create 2-3 random erasing patches
    #     for _ in range(2):
    #         patch_h = np.random.randint(oh // 10, oh // 6)
    #         patch_w = np.random.randint(ow // 10, ow // 6)
    #         y = np.random.randint(0, oh - patch_h)
    #         x = np.random.randint(0, ow - patch_w)
    #         # Fill with random gray value (simulating occlusion)
    #         cutout_img[y:y+patch_h, x:x+patch_w] = np.random.rand()
    #     augmentations.append(('cutout', np.clip(cutout_img, 0, 1)))
    # except Exception as e:
    #     print(f"Warning: Cutout failed - {e}")

    # 9) COMBINED: Flip + Brightness (common real-world scenario)
    try:
        flip_bright = adjust_gamma(np.flip(img, axis=1), gamma=1.2)
        augmentations.append(('flip_h_bright', np.clip(flip_bright, 0, 1)))
    except Exception as e:
        print(f"Warning: Combined flip+bright failed - {e}")

    return augmentations


def extract_features_from_tile(tile_gray, tile_gray_u8, tile_rgb, include_orb=False, orb_detector=None):
    """
    Extract all features from a single tile.
    Returns dict with 'hog', 'lbp', 'color_histogram', and optionally 'orb' features.
    """
    features_dict = {}
    
    # HOG Features - Good for bat and stadium structure detection
    hog_feat = hog(
        tile_gray,
        orientations=9,
        pixels_per_cell=(8, 8),
        cells_per_block=(2, 2),
        block_norm='L2-Hys'
    )
    features_dict['hog'] = hog_feat
    
    # LBP Features - Good for texture patterns in stadium and field
    P = 8
    R = 1
    n_bins = P + 2
    lbp = local_binary_pattern(tile_gray_u8, P=P, R=R, method='uniform')
    hist_lbp, _ = np.histogram(lbp.ravel(), bins=n_bins, range=(0, n_bins), density=True)
    hist_lbp = (hist_lbp.astype('float')) / (hist_lbp.sum() + 1e-7)
    features_dict['lbp'] = hist_lbp
    
    # Color Histogram - Critical for distinguishing white/red ball, jerseys, green field
    hist_r, _ = np.histogram(tile_rgb[:, :, 0], bins=32, range=(0, 1))
    hist_g, _ = np.histogram(tile_rgb[:, :, 1], bins=32, range=(0, 1))
    hist_b, _ = np.histogram(tile_rgb[:, :, 2], bins=32, range=(0, 1))
    hist_rgb = np.concatenate([hist_r, hist_g, hist_b]).astype('float')
    hist_rgb /= (hist_rgb.sum() + 1e-7)
    features_dict['color_histogram'] = hist_rgb
    
    # ORB Features - Good for detecting distinctive features (ball, bat edges)
    if include_orb and orb_detector is not None:
        kps, desc = orb_detector.detectAndCompute(tile_gray_u8, None)
        if desc is None:
            desc = np.zeros((1, 32), dtype=np.uint8)
        if len(kps) > 0:
            num_kp = len(kps)
            resp = np.array([kp.response for kp in kps])
            resp_mean = float(np.mean(resp))
            resp_std = float(np.std(resp)) if len(resp) > 1 else 0.0
            desc_mean = np.mean(desc, axis=0)
            desc_std = np.std(desc, axis=0)
            orb_vec = np.concatenate([[num_kp, resp_mean, resp_std], desc_mean, desc_std])
        else:
            orb_vec = np.zeros(67)
        features_dict['orb'] = orb_vec
        features_dict['num_keypoints'] = len(kps)
    
    return features_dict


def extract_cricket_augmented_features(file, include_orb=False, orb_n_features=500):
    """
    Extract features from cricket images with cricket-specific augmentations.
    Optimized for bat, ball, and stadium detection.
    
    Parameters:
    - file: Image filename
    - include_orb: Whether to extract ORB features (recommended for small objects like ball)
    - orb_n_features: Number of ORB keypoints to detect
    """
    file_path = os.path.join(data_dir, file)
    
    # Check if file is an image
    if not file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
        print(f"Skipping non-image file: {file}")
        return
    
    try:
        image_rgb = imread(file_path)
    except Exception as e:
        print(f"Error reading {file}: {e}")
        return
    
    # Resize to standard size
    image_rgb = resize(image_rgb, (800, 600), anti_aliasing=True)
    
    # Apply cricket-specific augmentations
    augmented_images = apply_cricket_augmentations(image_rgb)
    
    # Initialize ORB detector once if needed
    orb_detector = cv2.ORB_create(nfeatures=orb_n_features) if include_orb else None
    
    print(f"Processing {file} with {len(augmented_images)} cricket-optimized augmentations...")
    
    # Process each augmentation
    for aug_name, aug_img in augmented_images:
        # Prepare image versions
        gray_image = rgb2gray(aug_img)
        gray_uint8 = (gray_image * 255).astype(np.uint8)
        
        # Get dimensions and calculate tile size
        height, width = gray_image.shape
        tile_height = height // 8
        tile_width = width // 8
        
        # Process each tile
        for i in range(8):
            for j in range(8):
                # Calculate boundaries
                y_start = i * tile_height
                y_end = (i + 1) * tile_height
                x_start = j * tile_width
                x_end = (j + 1) * tile_width
                
                # Extract tiles
                tile_gray = gray_image[y_start:y_end, x_start:x_end]
                tile_gray_u8 = gray_uint8[y_start:y_end, x_start:x_end]
                tile_rgb = aug_img[y_start:y_end, x_start:x_end]
                
                # Extract all features for this tile
                tile_features = extract_features_from_tile(
                    tile_gray, tile_gray_u8, tile_rgb, 
                    include_orb=include_orb, 
                    orb_detector=orb_detector
                )
                
                # Store features in global data list
                base_info = {
                    'image': file,
                    'tile_i': i,
                    'tile_j': j,
                    'augmentation': aug_name
                }
                
                # Add HOG
                data.append({**base_info, 'type': 'hog', 'features': tile_features['hog']})
                
                # Add LBP
                data.append({**base_info, 'type': 'lbp', 'features': tile_features['lbp']})
                
                # Add Color Histogram
                data.append({**base_info, 'type': 'color_histogram', 'features': tile_features['color_histogram']})
                
                # Add ORB if requested
                if include_orb:
                    data.append({
                        **base_info, 
                        'type': 'orb', 
                        'features': tile_features['orb'],
                        'num_keypoints': tile_features.get('num_keypoints', 0)
                    })
    
    print(f"✓ Completed cricket-augmented feature extraction for {file}")


def write_augmented_features_to_file(filename='features_data_cricket_augmented.csv', tagged_data_path='tagged_data_sushant.csv'):
    """
    Write augmented features to CSV with augmentation tracking and labels from tagged data.
    Groups by (image, tile_i, tile_j, augmentation).
    Matches labels from tagged_data_sushant.csv based on image_name, row, col.
    """
    # Load tagged data
    try:
        tagged_df = pd.read_csv(tagged_data_path)
        print(f"✓ Loaded tagged data from {tagged_data_path}")
        print(f"  Tagged data shape: {tagged_df.shape}")
        
        # Create a lookup dictionary for quick label retrieval
        # Key: (image_name, row, col) -> Value: y
        label_lookup = {}
        for _, row in tagged_df.iterrows():
            key = (row['image_name'], row['row'], row['col'])
            label_lookup[key] = row['y']
        
        print(f"  Created label lookup with {len(label_lookup)} entries")
    except Exception as e:
        print(f"⚠️  Warning: Could not load tagged data: {e}")
        print(f"  Proceeding without labels...")
        label_lookup = {}
    
    # Group data by (image, tile_i, tile_j, augmentation)
    grouped_data = {}
    
    for item in data:
        aug = item.get('augmentation', 'original')
        key = (item['image'], item['tile_i'], item['tile_j'], aug)
        if key not in grouped_data:
            grouped_data[key] = {
                'image': item['image'],
                'tile_i': item['tile_i'],
                'tile_j': item['tile_j'],
                'augmentation': aug,
                'hog': None,
                'lbp': None,
                'color_histogram': None,
                'orb': None
            }
        
        # Store features by type
        grouped_data[key][item['type']] = item['features']
    
    # Create records
    records = []
    matched_labels = 0
    unmatched_labels = 0
    
    for key, tile_data in grouped_data.items():
        record = {
            'image': tile_data['image'],
            'tile_i': tile_data['tile_i'],
            'tile_j': tile_data['tile_j'],
            'augmentation': tile_data['augmentation']
        }
        
        # Look up the label from tagged data
        # For original images, match directly; for augmentations, use same label as original
        lookup_key = (tile_data['image'], tile_data['tile_i'], tile_data['tile_j'])
        if lookup_key in label_lookup:
            record['y'] = label_lookup[lookup_key]
            matched_labels += 1
        else:
            record['y'] = None  # or use -1, 0, or np.nan as default
            unmatched_labels += 1
        
        # Add HOG features
        if tile_data['hog'] is not None:
            for idx, val in enumerate(tile_data['hog']):
                record[f'x_hog_{idx}'] = val
        
        # Add LBP features
        if tile_data['lbp'] is not None:
            for idx, val in enumerate(tile_data['lbp']):
                record[f'x_lbp_{idx}'] = val
        
        # Add Color Histogram features
        if tile_data['color_histogram'] is not None:
            for idx, val in enumerate(tile_data['color_histogram']):
                record[f'x_color_{idx}'] = val
        
        # Add ORB features
        if tile_data['orb'] is not None:
            for idx, val in enumerate(tile_data['orb']):
                record[f'x_orb_{idx}'] = val
        
        records.append(record)
    
    # Create DataFrame and save
    df = pd.DataFrame(records)
    
    # Reorder columns to have y right after augmentation
    cols = ['image', 'tile_i', 'tile_j', 'augmentation', 'y']
    feature_cols = [col for col in df.columns if col not in cols]
    df = df[cols + feature_cols]
    
    df.to_csv(filename, index=False)
    
    print(f'\n{"="*70}')
    print(f'✓ Saved {len(df)} cricket-augmented feature records to {filename}')
    print(f'  Shape: {df.shape}')
    print(f'  Augmentations: {len(df["augmentation"].unique())} types')
    print(f'  Images: {df["image"].nunique()} unique files')
    print(f'  Tiles per image: 64 (8x8 grid)')
    print(f'  Total records per image: {64 * len(df["augmentation"].unique())}')
    print(f'  Labels matched: {matched_labels} ({matched_labels/len(df)*100:.1f}%)')
    print(f'  Labels unmatched: {unmatched_labels} ({unmatched_labels/len(df)*100:.1f}%)')
    if unmatched_labels > 0:
        print(f'  ⚠️  {unmatched_labels} records have no label (set to None)')
    print(f'  Sample columns: {list(df.columns[:10])}...')
    print(f'{"="*70}\n')
    
    return df


# ============================================================================
# EXAMPLE USAGE - Run cricket-augmented feature extraction
# ============================================================================

# Clear existing data
data = []
data_dir = '../../../notebooks/sushant07/raw_image/test'
# Extract cricket-augmented features for all images
# Set include_orb=True for better ball detection (slower but more accurate)
for file in os.listdir(data_dir):
    extract_cricket_augmented_features(file, include_orb=False)

# Write to CSV
write_augmented_features_to_file(filename='features_data_augmented_test.csv')

print("✓ Cricket-optimized augmentation functions loaded!")
print("\n🏏 Cricket-Specific Augmentations Included:")
print("  CRITICAL:")
print("    - Horizontal flip (left/right-handed batsmen)")
print("    - Brightness variations (day/night matches)")
print("    - Contrast adjustments (lighting conditions)")
print("\n  IMPORTANT:")
print("    - Small rotations (±5-8° camera angles)")
print("    - Color jitter (different cameras/feeds)")
print("    - Gaussian noise (broadcast compression)")
print("\n  CRICKET-SPECIFIC:")
print("    - Random crop (partial stadium/zoomed shots)")
print("    - Cutout/occlusion (players blocking view)")
print("    - Combined augmentations (flip+bright)")
print("\n⚠️  EXCLUDED:")
print("    - Zoom/Scale (removed as requested)")
print("    - Motion blur (removed as requested)")
print("    - Vertical flip (not suitable for cricket)")
print("    - Heavy rotation >15° (unrealistic)")
print("\nUncomment the example usage section to run extraction.")
print("Recommended: Set include_orb=True for better ball detection.")

Processing pexels.jpg with 7 cricket-optimized augmentations...
✓ Completed cricket-augmented feature extraction for pexels.jpg
✓ Loaded tagged data from tagged_data_sushant.csv
  Tagged data shape: (6464, 5)
  Created label lookup with 6464 entries

✓ Saved 448 cricket-augmented feature records to features_data_augmented_test.csv
  Shape: (448, 3279)
  Augmentations: 7 types
  Images: 1 unique files
  Tiles per image: 64 (8x8 grid)
  Total records per image: 448
  Labels matched: 0 (0.0%)
  Labels unmatched: 448 (100.0%)
  ⚠️  448 records have no label (set to None)
  Sample columns: ['image', 'tile_i', 'tile_j', 'augmentation', 'y', 'x_hog_0', 'x_hog_1', 'x_hog_2', 'x_hog_3', 'x_hog_4']...

✓ Cricket-optimized augmentation functions loaded!

🏏 Cricket-Specific Augmentations Included:
  CRITICAL:
    - Horizontal flip (left/right-handed batsmen)
    - Brightness variations (day/night matches)
    - Contrast adjustments (lighting conditions)

  IMPORTANT:
    - Small rotations (±5-8° c